# Introduction to atomman: Unit Conversions

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.

Notebook last updated: 2018-04-05
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction

The atomman.unitconvert submodule includes tools for handling unit conversions within calculations. The unitconvert module is built around the [numericalunits](https://pypi.python.org/pypi/numericalunits) package, extending it with useful functions and tools.

Units are handled in the following manner:

1. Parameters are 'set' using functions that take value(s) and unit fields. The functions convert the values to common working units.

2. All calculations are performed in the compatible working units.

3. When finished, 'get' functions convert values from the working units to whatever units you want.

Note that units are *not* tracked throughout the calculation, only conversions are performed at the beginning and end.  This is advantageous as calculations and functions can be implemented without caring about the units, and there is no extra overhead.  The disadvantage is that there is no explicit checking of compatible conversions, although implicit checking is possible (see [Section #4](#section4), or the [numericalunits](https://pypi.python.org/pypi/numericalunits) documentation.)

**Library Imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.unitconvert as uc

# Show atomman version
print('atomman version =', am.__version__)

atomman version = 1.2.alpha


## 2. Basics

### 2.1 unit dictionary

The unit dictionary stores all units defined by [numericalunits](https://pypi.python.org/pypi/numericalunits). This keeps the namespace clean and allows for units to be accessed by string.

In [2]:
print(list(uc.unit.keys()))

['pi', 'm', 'kg', 's', 'C', 'K', 'cm', 'mm', 'um', 'nm', 'pm', 'fm', 'km', 'angstrom', 'lightyear', 'astro_unit', 'pc', 'kpc', 'Mpc', 'Gpc', 'inch', 'foot', 'mile', 'thou', 'L', 'mL', 'uL', 'nL', 'pL', 'fL', 'aL', 'kL', 'ML', 'GL', 'ms', 'us', 'ns', 'ps', 'fs', 'minute', 'hour', 'day', 'week', 'year', 'Hz', 'mHz', 'kHz', 'MHz', 'GHz', 'THz', 'PHz', 'g', 'mg', 'ug', 'ng', 'pg', 'fg', 'tonne', 'amu', 'Da', 'kDa', 'lbm', 'J', 'mJ', 'uJ', 'nJ', 'pJ', 'fJ', 'kJ', 'MJ', 'GJ', 'erg', 'eV', 'meV', 'keV', 'MeV', 'GeV', 'TeV', 'btu', 'smallcal', 'kcal', 'Wh', 'kWh', 'NA', 'mol', 'mmol', 'umol', 'nmol', 'pmol', 'fmol', 'M', 'mM', 'uM', 'nM', 'pM', 'fM', 'N', 'mN', 'uN', 'nN', 'pN', 'fN', 'kN', 'MN', 'GN', 'dyn', 'lbf', 'Pa', 'hPa', 'kPa', 'MPa', 'GPa', 'bar', 'mbar', 'cbar', 'dbar', 'kbar', 'Mbar', 'atm', 'torr', 'mtorr', 'psi', 'W', 'mW', 'uW', 'nW', 'pW', 'kW', 'MW', 'GW', 'TW', 'degFinterval', 'degCinterval', 'mK', 'uK', 'nK', 'pK', 'mC', 'uC', 'nC', 'Ah', 'mAh', 'A', 'mA', 'uA', 'nA', 'pA', '

### 2.2 Working units

By default, atomman defines working units in:
    
- length = 'angstrom' = 'Å'

- mass = 'amu'

- energy = 'eV'

- charge = 'e'

- temperature = 'K'

All other units are derived relative to these.

In [3]:
print('angstrom =', uc.unit['angstrom'])
print('amu =     ', uc.unit['amu'])
print('eV =      ', uc.unit['eV'])
print('e =       ', uc.unit['e'])
print('K =       ', uc.unit['K'])
print('nm =      ', uc.unit['nm'])
print('g =       ', uc.unit['g'])
print('J =       ', uc.unit['J'])
print('ps =      ', uc.unit['ps'])

angstrom = 1.0
amu =      1.0
eV =       1.0
e =        1.0
K =        1.0
nm =       10.0
g =        6.022141290116741e+23
J =        6.241509343260179e+18
ps =       98.22694969377756


### 2.3 reset_units()

The working units can be altered using reset_units(). You can specify up to four out of five of length, mass, time, energy, and charge. If less than four values are given, SI units are used. Temperature is always 'K' when values are specified.

In [4]:
# Reset working units such that length is in 'nm', mass is in 'g' and time is in 'ps'
uc.reset_units(length='nm', mass='g', time='ps')

print('angstrom =', uc.unit['angstrom'])
print('amu =     ', uc.unit['amu'])
print('eV =      ', uc.unit['eV'])
print('e =       ', uc.unit['e'])
print('K =       ', uc.unit['K'])
print('nm =      ', uc.unit['nm'])
print('g =       ', uc.unit['g'])
print('J =       ', uc.unit['J'])
print('ps =      ', uc.unit['ps'])

angstrom = 0.09999999999999999
amu =      1.660538921e-24
eV =       1.6021765649999997e-22
e =        1.602176565e-19
K =        1.0
nm =       0.9999999999999999
g =        1.0
J =        0.0009999999999999998
ps =       1.0


Alternatively, if you call reset_units without arguments it will use the default numericalunits option and generate random working units.  This can be useful for debugging code (see [Section #4](#section4), or the [numericalunits](https://pypi.python.org/pypi/numericalunits) documentation).

In [5]:
# Reset working units to random values
uc.reset_units()

print('angstrom =', uc.unit['angstrom'])
print('amu =     ', uc.unit['amu'])
print('eV =      ', uc.unit['eV'])
print('e =       ', uc.unit['e'])
print('K =       ', uc.unit['K'])
print('nm =      ', uc.unit['nm'])
print('g =       ', uc.unit['g'])
print('J =       ', uc.unit['J'])
print('ps =      ', uc.unit['ps'])

angstrom = 4.326483125589462e-10
amu =      1.1521016888004465e-27
eV =       3.404834648620962e-17
e =        4.178487729967224e-20
K =        4.714945128643944
nm =       4.326483125589462e-09
g =        0.0006938119150538397
J =        212.51307271623725
ps =       2.4720831005255104e-13


In [6]:
# Return working units to atomman's default
uc.reset_units(length = 'angstrom', mass = 'amu', energy='eV', charge='e')

### 2.4 Setting and getting static values

Static numerical values can be set and get in one of two ways:

- set by multiplying value by units, and get by dividing by units.

- use the set_in_units() and get_in_units() functions.

#### 2.4.1 Direct setting and getting

In [7]:
# Convert volume from angstrom^3 to nm^3
print('10 angstrom^3 =')
volume = 10 * uc.unit['angstrom']**3

print(volume / uc.unit['nm']**3, 'nm^3')

10 angstrom^3 =
0.01 nm^3


In [8]:
# Show Pa = kg/(m*s^2)
print('5.5 kg/(m*s^2) =')
pressure = 5.5 * uc.unit['kg'] / (uc.unit['m']*uc.unit['s']**2)

print(pressure / uc.unit['Pa'], 'Pa')

5.5 kg/(m*s^2) =
5.5 Pa


In [9]:
# Show that conversions work with arrays
stress = np.array([[1.1, 1.2, 1.3],
                   [1.2, 2.2, 2.3],
                   [1.3, 2.3, 3.3]]) * uc.unit['GPa']

print(stress / uc.unit['MPa'], 'MPa')

[[ 1100.  1200.  1300.]
 [ 1200.  2200.  2300.]
 [ 1300.  2300.  3300.]] MPa


#### 2.4.2 parse()

As the above example shows, expressing complex units can get messy and unclear. The parse() function makes this easier by allowing complex units to be parsed from strings.

In [10]:
# Convert volume from angstrom^3 to nm^3
print('10 angstrom^3 =')
volume = 10 * uc.parse('angstrom^3')

print(volume / uc.parse('nm^3'), 'nm^3')

10 angstrom^3 =
0.01 nm^3


In [11]:
# Show Pa = kg/(m*s^2)
print('5.5 kg/(m*s^2) =')
pressure = 5.5 * uc.parse('kg/(m*s^2)')

print(pressure / uc.parse('Pa'), 'Pa')

5.5 kg/(m*s^2) =
5.499999999999999 Pa


#### 2.4.3 set_in_units() and get_in_units()

Both functions take a value and a unit string, call parse on the unit string and perform the correct * or /.

In [12]:
# Convert volume from angstrom^3 to nm^3
print('10 angstrom^3 =')
volume = uc.set_in_units(10, 'angstrom^3')

print(uc.get_in_units(volume, 'nm^3'), 'nm^3')

10 angstrom^3 =
0.01 nm^3


In [13]:
# Show Pa = kg/(m*s^2)
print('5.5 kg/(m*s^2) =')
pressure = uc.set_in_units(5.5, 'kg/(m*s^2)')

print(uc.get_in_units(pressure, 'Pa'), 'Pa')

5.5 kg/(m*s^2) =
5.5 Pa


In [14]:
# Show that conversions work with arrays
stress = uc.set_in_units(np.array([[1.1, 1.2, 1.3],
                                   [1.2, 2.2, 2.3],
                                   [1.3, 2.3, 3.3]]), 'GPa')

print(uc.get_in_units(stress, 'MPa'), 'MPa')

[[ 1100.  1200.  1300.]
 [ 1200.  2200.  2300.]
 [ 1300.  2300.  3300.]] MPa


### 2.5 set_literal()

Values can also be read in from strings with set_literal().

In [15]:
# Convert volume from angstrom^3 to nm^3
print('10 angstrom^3 =')
volume = uc.set_literal('10 angstrom^3')

print(uc.get_in_units(volume, 'nm^3'), 'nm^3')

10 angstrom^3 =
0.01 nm^3


In [16]:
# Show that conversions work with arrays
stress = uc.set_literal("""[[1.1, 1.2, 1.3], 
                            [1.2, 2.2, 2.3], 
                            [1.3, 2.3, 3.3]] GPa""")

print(uc.get_in_units(stress, 'MPa'), 'MPa')

[[ 1100.  1200.  1300.]
 [ 1200.  2200.  2300.]
 [ 1300.  2300.  3300.]] MPa


## 3. Data model representations

In addition to the basic conversions, unitconvert also allows for the values to be returned as and extracted from a DataModelDict. This provides a means of representing the data equivalently in either JSON or XML.

### 3.1 model()

Values can be converted into a structured data model using model().

In [17]:
# Set length as 4 nm
length = uc.set_in_units(4, 'nm')

# Transform length into a model with units in angstrom
lmodel = uc.model(length, 'angstrom')

# Print lmodel as XML
print(lmodel.xml(full_document=False))

<value>40.0</value><unit>angstrom</unit>


In [18]:
# Set list of temperatures in K
temperatures = uc.set_in_units([10,20,30,40,50], 'K')

# Transform temperatures into a model with units in K
tmodel = uc.model(temperatures, 'K')

# Print tmodel as JSON
print(tmodel.json(indent=2))

{
  "value": [
    10.0, 
    20.0, 
    30.0, 
    40.0, 
    50.0
  ], 
  "unit": "K"
}


For equivalent JSON/XML representation, values with 2 or more dimensions are flattened and the shape is included in the model.

In [19]:
# Set stress tensor in 'MPa'
stress = uc.set_in_units(np.array([[1.1, 0.0, 0.0],
                                   [0.0, 2.0, 0.5],
                                   [0.0, 0.5, -1.4]]), 'MPa')
                         
# Transform stress into a model with units in kPa
smodel = uc.model(stress, 'kPa')

# Print smodel as JSON
print(smodel.json())
print()

# Print smodel as XML
print(smodel.xml(full_document=False))

{"value": [1100.0, 0.0, 0.0, 0.0, 2000.0, 500.0, 0.0, 500.0, -1399.9999999999998], "shape": [3, 3], "unit": "kPa"}

<value>1100.0</value><value>0.0</value><value>0.0</value><value>0.0</value><value>2000.0</value><value>500.0</value><value>0.0</value><value>500.0</value><value>-1399.9999999999998</value><shape>3</shape><shape>3</shape><unit>kPa</unit>


### 3.2 value_unit()

Values can then be read back in from a model, XML or JSON using value_unit().

In [20]:
# Read lmode to set length
length = uc.value_unit(lmodel)

# Print length in nm
print(uc.get_in_units(length, 'nm'), 'nm')

4.0 nm


In [21]:
# Read tmodel to set temperatures 
temperatures = uc.value_unit(tmodel)

# Print temperatures in K
print(uc.get_in_units(temperatures, 'K'), 'K')

[ 10.  20.  30.  40.  50.] K


In [22]:
# Read smodel to set stress 
stress = uc.value_unit(smodel)

# Print stress in 'MPa'
print(uc.get_in_units(stress, 'MPa'), 'MPa')

[[ 1.1  0.   0. ]
 [ 0.   2.   0.5]
 [ 0.   0.5 -1.4]] MPa


### 3.3 error_unit()

Standard errors associated with each given value can also be included in the model, which can then be retrieved using error_unit().

In [23]:
# Generate realistic-looking nonsense
xcoordinate = np.array([1, 2, 3, 4, 5]) + 0.2 * np.random.rand(5) - 0.1
xcoorderror = np.array([0.2, 0.2, 0.2, 0.2, 0.2]) + 0.02 * np.random.rand(5) - 0.01

# Assign units to nonsense
xcoordinate = uc.set_in_units(xcoordinate, 'cm')
xcoorderror = uc.set_in_units(xcoorderror, 'cm')

# Generate model of nonsense with error
model = uc.model(xcoordinate, 'm', error=xcoorderror)
print(model.json(indent=2))

{
  "value": [
    0.00938526143366722, 
    0.019963016186483936, 
    0.029753864372104227, 
    0.04003535935351832, 
    0.049325127074855736
  ], 
  "error": [
    0.002099727514227793, 
    0.0020338416998994546, 
    0.002011570663077151, 
    0.0019304454834472872, 
    0.00208441148464533
  ], 
  "unit": "m"
}


Errors can then be similarly extracted from the model using error_unit()

In [24]:
# Read realistic-looking nonsense back in
print('value =', uc.get_in_units(uc.value_unit(model), 'mm'), 'mm')
print('error =', uc.get_in_units(uc.error_unit(model), 'mm'), 'mm')

value = [  9.38526143  19.96301619  29.75386437  40.03535935  49.32512707] mm
error = [ 2.09972751  2.0338417   2.01157066  1.93044548  2.08441148] mm


## 4. Unit debugging <a id='section4'></a>

There is no explicit unit control with unitconvert, but correct unit conversions can still be debugged and tested by seeing if changing the working units changes values.

In [25]:
# Print valid conversion
print('57 atm =', end=' ')
time = uc.set_in_units(57, 'atm')
print(uc.get_in_units(time, 'GPa'), 'GPa')

# Reset working units to random values
uc.reset_units()

# Print valid conversion again showing same results
print('57 atm =', end=' ')
time = uc.set_in_units(57, 'atm')
print(uc.get_in_units(time, 'GPa'), 'GPa')

57 atm = 0.005775525 GPa
57 atm = 0.005775525 GPa


In [26]:
# Print invalid conversion
print('57 s =', end=' ')
time = uc.set_in_units(57, 's')
print(uc.get_in_units(time, 'GPa'), 'GPa')

# Reset working units to random values
uc.reset_units()

# Print invalid conversion again showing different results
print('57 s =', end=' ')
time = uc.set_in_units(57, 's')
print(uc.get_in_units(time, 'GPa'), 'GPa')

57 s = 1.24582079917e-06 GPa
57 s = 4.58839169445e-07 GPa
